# Process sequencing-based neutralization assay plate
This notebook is designed to be run using `snakemake`.

## Setup
Import Python modules:

In [1]:
import altair as alt

import neutcurve

import pandas as pd

_ = alt.data_transformers.disable_max_rows()

Get the variables passed by `snakemake`:

In [2]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/seqneut-pipeline/lib/python3.11/site-packages', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmpseolozjc/file/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xbf\x89\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c,results/barcode_counts/plate11_D041d0_20.csv\x94\x8c,results/barcode_counts/plate11_D041d0_60.csv\x94\x8c-results/barcode_counts/plate11_D041d0_180.csv\x94\x8c-results/barcode_counts/plate11_D041d0_540.csv\x94\x8c.results/barcode_counts/plate11_D041d0_1620.csv\x94\x8c.results/barcode_counts/plate11_D041d0_4860.csv\x94\x8c/results/barcode_counts/plate11_D041d0_14580.csv\x94\x8c/results/barcode_counts/plate11_D041d0_43740.csv\x94\x8c0results/barcode_counts/plate11_D041d0_131220.csv\x94\x8c0results/barcode_counts/plate11_D041d0_393660.csv\x94\x8c.results/barcode_counts/plate11_D041d182_20.csv\x94\x8c.results/barcode_counts/plate11_D041d182_60.csv\x94\x8c/results/barcode_counts/plate11_D041d182_180.csv\x94\x8c/results/barcode_counts/plate11_D041d182_540.csv\x94\x8c0results/barcode_counts/plate11_D041d182_1620.csv\x94\x8c0results/barcode_counts/plate11_D041d182_4860.csv\x94\x8c1results/barcode_counts/plate11_D041d182_14580.csv\x94\x8c1results/barcode_counts/plate11_D041d182_43740.csv\x94\x8c2results/barcode_counts/plate11_D041d182_131220.csv\x94\x8c2results/barcode_counts/plate11_D041d182_393660.csv\x94\x8c-results/barcode_counts/plate11_D041d30_20.csv\x94\x8c-results/barcode_counts/plate11_D041d30_60.csv\x94\x8c.results/barcode_counts/plate11_D041d30_180.csv\x94\x8c.results/barcode_counts/plate11_D041d30_540.csv\x94\x8c/results/barcode_counts/plate11_D041d30_1620.csv\x94\x8c/results/barcode_counts/plate11_D041d30_4860.csv\x94\x8c0results/barcode_counts/plate11_D041d30_14580.csv\x94\x8c0results/barcode_counts/plate11_D041d30_43740.csv\x94\x8c1results/barcode_counts/plate11_D041d30_131220.csv\x94\x8c1results/barcode_counts/plate11_D041d30_393660.csv\x94\x8c,results/barcode_counts/plate11_M099d0_20.csv\x94\x8c,results/barcode_counts/plate11_M099d0_60.csv\x94\x8c-results/barcode_counts/plate11_M099d0_180.csv\x94\x8c-results/barcode_counts/plate11_M099d0_540.csv\x94\x8c.results/barcode_counts/plate11_M099d0_1620.csv\x94\x8c.results/barcode_counts/plate11_M099d0_4860.csv\x94\x8c/results/barcode_counts/plate11_M099d0_14580.csv\x94\x8c/results/barcode_counts/plate11_M099d0_43740.csv\x94\x8c0results/barcode_counts/plate11_M099d0_131220.csv\x94\x8c0results/barcode_counts/plate11_M099d0_393660.csv\x94\x8c.results/barcode_counts/plate11_M099d182_20.csv\x94\x8c.results/barcode_counts/plate11_M099d182_60.csv\x94\x8c/results/barcode_counts/plate11_M099d182_180.csv\x94\x8c/results/barcode_counts/plate11_M099d182_540.csv\x94\x8c0results/barcode_counts/plate11_M099d182_1620.csv\x94\x8c0results/barcode_counts/plate11_M099d182_4860.csv\x94\x8c1results/barcode_counts/plate11_M099d182_14580.csv\x94\x8c1results/barcode_counts/plate11_M099d182_43740.csv\x94\x8c2results/barcode_counts/plate11_M099d182_131220.csv\x94\x8c2results/barcode_counts/plate11_M099d182_393660.csv\x94\x8c-results/barcode_counts/plate11_M099d30_20.csv\x94\x8c-results/barcode_counts/plate11_M099d30_60.csv\x94\x8c.results/barcode_counts/plate11_M099d30_180.csv\x94\x8c.results/barcode_counts/plate11_M099d30_540.csv\x94\x8c/results/barcode_counts/plate11_M099d30_1620.csv\x94\x8c/results/barcode_counts/plate11_M099d30_4860.csv\x94\x8c0results/barcode_counts/plate11_M099d30_14580.csv\x94\x8c0results/barcode_counts/plate11_M099d30_43740.csv\x94\x8c1results/barcode_counts/plate11_M099d30_131220.csv\x94\x8c1results/barcode_counts/plate11_M099d30_393660.csv\x94\x8c,results/barcode_counts/plate11_Y044d0_20.csv\x94\x8c,results/barcode_counts/plate11_Y044d0_60.csv\x94\x8c-results/barcode_counts/plate11_Y044d0_180.csv\x94\x8c-results/barcode_counts/plate11_Y044d0_540.csv\x94\x8c.results/barcode_counts/plate11_Y044d0_1620.csv\x94\x8c.results/barcode_counts/plate11_Y044d0_4860.csv\x94\x8c/results/barcode_counts/plate11_Y044d0_14580.csv\x94\x8c/results/barcode_counts/plate11_Y044d0_43740.csv\x94\x8c0results/barcode_counts/plate11_Y044d0_131220.csv\x94\x8c0results/barcode_counts/plate11_Y044d0_393660.csv\x94\x8c-results/barcode_counts/plate11_Y044d30_20.csv\x94\x8c-results/barcode_counts/plate11_Y044d30_60.csv\x94\x8c.results/barcode_counts/plate11_Y044d30_180.csv\x94\x8c.results/barcode_counts/plate11_Y044d30_540.csv\x94\x8c/results/barcode_counts/plate11_Y044d30_1620.csv\x94\x8c/results/barcode_counts/plate11_Y044d30_4860.csv\x94\x8c0results/barcode_counts/plate11_Y044d30_14580.csv\x94\x8c0results/barcode_counts/plate11_Y044d30_43740.csv\x94\x8c1results/barcode_counts/plate11_Y044d30_131220.csv\x94\x8c1results/barcode_counts/plate11_Y044d30_393660.csv\x94\x8c)results/barcode_counts/plate11_none-2.csv\x94\x8c)results/barcode_counts/plate11_none-3.csv\x94\x8c)results/barcode_counts/plate11_none-4.csv\x94\x8c)results/barcode_counts/plate11_none-5.csv\x94\x8c)results/barcode_counts/plate11_none-6.csv\x94\x8c)results/barcode_counts/plate11_none-7.csv\x94\x8c)results/barcode_counts/plate11_none-8.csv\x94\x8c)results/barcode_counts/plate11_none-9.csv\x94\x8c*results/barcode_counts/plate11_none-10.csv\x94\x8c*results/barcode_counts/plate11_none-11.csv\x94\x8c*results/barcode_counts/plate11_none-12.csv\x94\x8c*results/barcode_counts/plate11_none-13.csv\x94\x8c*results/barcode_counts/plate11_none-14.csv\x94\x8c*results/barcode_counts/plate11_none-15.csv\x94\x8c*results/barcode_counts/plate11_none-16.csv\x94\x8c*results/barcode_counts/plate11_none-17.csv\x94\x8c+results/barcode_fates/plate11_D041d0_20.csv\x94\x8c+results/barcode_fates/plate11_D041d0_60.csv\x94\x8c,results/barcode_fates/plate11_D041d0_180.csv\x94\x8c,results/barcode_fates/plate11_D041d0_540.csv\x94\x8c-results/barcode_fates/plate11_D041d0_1620.csv\x94\x8c-results/barcode_fates/plate11_D041d0_4860.csv\x94\x8c.results/barcode_fates/plate11_D041d0_14580.csv\x94\x8c.results/barcode_fates/plate11_D041d0_43740.csv\x94\x8c/results/barcode_fates/plate11_D041d0_131220.csv\x94\x8c/results/barcode_fates/plate11_D041d0_393660.csv\x94\x8c-results/barcode_fates/plate11_D041d182_20.csv\x94\x8c-results/barcode_fates/plate11_D041d182_60.csv\x94\x8c.results/barcode_fates/plate11_D041d182_180.csv\x94\x8c.results/barcode_fates/plate11_D041d182_540.csv\x94\x8c/results/barcode_fates/plate11_D041d182_1620.csv\x94\x8c/results/barcode_fates/plate11_D041d182_4860.csv\x94\x8c0results/barcode_fates/plate11_D041d182_14580.csv\x94\x8c0results/barcode_fates/plate11_D041d182_43740.csv\x94\x8c1results/barcode_fates/plate11_D041d182_131220.csv\x94\x8c1results/barcode_fates/plate11_D041d182_393660.csv\x94\x8c,results/barcode_fates/plate11_D041d30_20.csv\x94\x8c,results/barcode_fates/plate11_D041d30_60.csv\x94\x8c-results/barcode_fates/plate11_D041d30_180.csv\x94\x8c-results/barcode_fates/plate11_D041d30_540.csv\x94\x8c.results/barcode_fates/plate11_D041d30_1620.csv\x94\x8c.results/barcode_fates/plate11_D041d30_4860.csv\x94\x8c/results/barcode_fates/plate11_D041d30_14580.csv\x94\x8c/results/barcode_fates/plate11_D041d30_43740.csv\x94\x8c0results/barcode_fates/plate11_D041d30_131220.csv\x94\x8c0results/barcode_fates/plate11_D041d30_393660.csv\x94\x8c+results/barcode_fates/plate11_M099d0_20.csv\x94\x8c+results/barcode_fates/plate11_M099d0_60.csv\x94\x8c,results/barcode_fates/plate11_M099d0_180.csv\x94\x8c,results/barcode_fates/plate11_M099d0_540.csv\x94\x8c-results/barcode_fates/plate11_M099d0_1620.csv\x94\x8c-results/barcode_fates/plate11_M099d0_4860.csv\x94\x8c.results/barcode_fates/plate11_M099d0_14580.csv\x94\x8c.results/barcode_fates/plate11_M099d0_43740.csv\x94\x8c/results/barcode_fates/plate11_M099d0_131220.csv\x94\x8c/results/barcode_fates/plate11_M099d0_393660.csv\x94\x8c-results/barcode_fates/plate11_M099d182_20.csv\x94\x8c-results/barcode_fates/plate11_M099d182_60.csv\x94\x8c.results/barcode_fates/plate11_M099d182_180.csv\x94\x8c.results/barcode_fates/plate11_M099d182_540.csv\x94\x8c/results/barcode_fates/plate11_M099d182_1620.csv\x94\x8c/results/barcode_fates/plate11_M099d182_4860.csv\x94\x8c0results/barcode_fates/plate11_M099d182_14580.csv\x94\x8c0results/barcode_fates/plate11_M099d182_43740.csv\x94\x8c1results/barcode_fates/plate11_M099d182_131220.csv\x94\x8c1results/barcode_fates/plate11_M099d182_393660.csv\x94\x8c,results/barcode_fates/plate11_M099d30_20.csv\x94\x8c,results/barcode_fates/plate11_M099d30_60.csv\x94\x8c-results/barcode_fates/plate11_M099d30_180.csv\x94\x8c-results/barcode_fates/plate11_M099d30_540.csv\x94\x8c.results/barcode_fates/plate11_M099d30_1620.csv\x94\x8c.results/barcode_fates/plate11_M099d30_4860.csv\x94\x8c/results/barcode_fates/plate11_M099d30_14580.csv\x94\x8c/results/barcode_fates/plate11_M099d30_43740.csv\x94\x8c0results/barcode_fates/plate11_M099d30_131220.csv\x94\x8c0results/barcode_fates/plate11_M099d30_393660.csv\x94\x8c+results/barcode_fates/plate11_Y044d0_20.csv\x94\x8c+results/barcode_fates/plate11_Y044d0_60.csv\x94\x8c,results/barcode_fates/plate11_Y044d0_180.csv\x94\x8c,results/barcode_fates/plate11_Y044d0_540.csv\x94\x8c-results/barcode_fates/plate11_Y044d0_1620.csv\x94\x8c-results/barcode_fates/plate11_Y044d0_4860.csv\x94\x8c.results/barcode_fates/plate11_Y044d0_14580.csv\x94\x8c.results/barcode_fates/plate11_Y044d0_43740.csv\x94\x8c/results/barcode_fates/plate11_Y044d0_131220.csv\x94\x8c/results/barcode_fates/plate11_Y044d0_393660.csv\x94\x8c,results/barcode_fates/plate11_Y044d30_20.csv\x94\x8c,results/barcode_fates/plate11_Y044d30_60.csv\x94\x8c-results/barcode_fates/plate11_Y044d30_180.csv\x94\x8c-results/barcode_fates/plate11_Y044d30_540.csv\x94\x8c.results/barcode_fates/plate11_Y044d30_1620.csv\x94\x8c.results/barcode_fates/plate11_Y044d30_4860.csv\x94\x8c/results/barcode_fates/plate11_Y044d30_14580.csv\x94\x8c/results/barcode_fates/plate11_Y044d30_43740.csv\x94\x8c0results/barcode_fates/plate11_Y044d30_131220.csv\x94\x8c0results/barcode_fates/plate11_Y044d30_393660.csv\x94\x8c(results/barcode_fates/plate11_none-2.csv\x94\x8c(results/barcode_fates/plate11_none-3.csv\x94\x8c(results/barcode_fates/plate11_none-4.csv\x94\x8c(results/barcode_fates/plate11_none-5.csv\x94\x8c(results/barcode_fates/plate11_none-6.csv\x94\x8c(results/barcode_fates/plate11_none-7.csv\x94\x8c(results/barcode_fates/plate11_none-8.csv\x94\x8c(results/barcode_fates/plate11_none-9.csv\x94\x8c)results/barcode_fates/plate11_none-10.csv\x94\x8c)results/barcode_fates/plate11_none-11.csv\x94\x8c)results/barcode_fates/plate11_none-12.csv\x94\x8c)results/barcode_fates/plate11_none-13.csv\x94\x8c)results/barcode_fates/plate11_none-14.csv\x94\x8c)results/barcode_fates/plate11_none-15.csv\x94\x8c)results/barcode_fates/plate11_none-16.csv\x94\x8c)results/barcode_fates/plate11_none-17.csv\x94\x8c-data/viral_libraries/pdmH1N1_lib2023_loes.csv\x94\x8c3data/neut_standard_sets/loes2023_neut_standards.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\ncount_csvs\x94K\x00K`\x86\x94\x8c\tfate_csvs\x94K`K\xc0\x86\x94\x8c\x11viral_library_csv\x94K\xc0N\x86\x94\x8c\x15neut_standard_set_csv\x94K\xc1N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\xd9\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\xdf)}\x94\x8c\x05_name\x94h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bh\xcfh\x06\x8c\tNamedlist\x94\x93\x94)\x81\x94(h\nh\x0bh\x0ch\rh\x0eh\x0fh\x10h\x11h\x12h\x13h\x14h\x15h\x16h\x17h\x18h\x19h\x1ah\x1bh\x1ch\x1dh\x1eh\x1fh h!h"h#h$h%h&h\'h(h)h*h+h,h-h.h/h0h1h2h3h4h5h6h7h8h9h:h;h<h=h>h?h@hAhBhChDhEhFhGhHhIhJhKhLhMhNhOhPhQhRhShThUhVhWhXhYhZh[h\\h]h^h_h`hahbhchdhehfhghhhie}\x94(h\xcd}\x94h\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bubh\xd1h\xea)\x81\x94(hjhkhlhmhnhohphqhrhshthuhvhwhxhyhzh{h|h}h~h\x7fh\x80h\x81h\x82h\x83h\x84h\x85h\x86h\x87h\x88h\x89h\x8ah\x8bh\x8ch\x8dh\x8eh\x8fh\x90h\x91h\x92h\x93h\x94h\x95h\x96h\x97h\x98h\x99h\x9ah\x9bh\x9ch\x9dh\x9eh\x9fh\xa0h\xa1h\xa2h\xa3h\xa4h\xa5h\xa6h\xa7h\xa8h\xa9h\xaah\xabh\xach\xadh\xaeh\xafh\xb0h\xb1h\xb2h\xb3h\xb4h\xb5h\xb6h\xb7h\xb8h\xb9h\xbah\xbbh\xbch\xbdh\xbeh\xbfh\xc0h\xc1h\xc2h\xc3h\xc4h\xc5h\xc6h\xc7h\xc8h\xc9e}\x94(h\xcd}\x94h\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bubh\xd3h\xcah\xd5h\xcbub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94(\x8c&results/plates/plate11/qc_failures.txt\x94\x8c+results/plates/plate11/frac_infectivity.csv\x94e}\x94(h\xcd}\x94(\x8c\x0bqc_failures\x94K\x00N\x86\x94\x8c\x14frac_infectivity_csv\x94K\x01N\x86\x94uh\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bj\x0b\x01\x00\x00j\x07\x01\x00\x00j\r\x01\x00\x00j\x08\x01\x00\x00ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94(\x8c\x12pandas.core.series\x94\x8c\x06Series\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x12SingleBlockManager\x94\x93\x94)\x81\x94(]\x94\x8c\x18pandas.core.indexes.base\x94\x8c\n_new_Index\x94\x93\x94\x8c\x19pandas.core.indexes.range\x94\x8c\nRangeIndex\x94\x93\x94}\x94(\x8c\x04name\x94N\x8c\x05start\x94K\x00\x8c\x04stop\x94K`\x8c\x04step\x94K\x01u\x86\x94R\x94a]\x94\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K`\x85\x94j8\x01\x00\x00\x8c\x05dtype\x94\x93\x94\x8c\x02O8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01|\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK?t\x94b\x89]\x94(\x8c\x11plate11_D041d0_20\x94\x8c\x11plate11_D041d0_60\x94\x8c\x12plate11_D041d0_180\x94\x8c\x12plate11_D041d0_540\x94\x8c\x13plate11_D041d0_1620\x94\x8c\x13plate11_D041d0_4860\x94\x8c\x14plate11_D041d0_14580\x94\x8c\x14plate11_D041d0_43740\x94\x8c\x15plate11_D041d0_131220\x94\x8c\x15plate11_D041d0_393660\x94\x8c\x13plate11_D041d182_20\x94\x8c\x13plate11_D041d182_60\x94\x8c\x14plate11_D041d182_180\x94\x8c\x14plate11_D041d182_540\x94\x8c\x15plate11_D041d182_1620\x94\x8c\x15plate11_D041d182_4860\x94\x8c\x16plate11_D041d182_14580\x94\x8c\x16plate11_D041d182_43740\x94\x8c\x17plate11_D041d182_131220\x94\x8c\x17plate11_D041d182_393660\x94\x8c\x12plate11_D041d30_20\x94\x8c\x12plate11_D041d30_60\x94\x8c\x13plate11_D041d30_180\x94\x8c\x13plate11_D041d30_540\x94\x8c\x14plate11_D041d30_1620\x94\x8c\x14plate11_D041d30_4860\x94\x8c\x15plate11_D041d30_14580\x94\x8c\x15plate11_D041d30_43740\x94\x8c\x16plate11_D041d30_131220\x94\x8c\x16plate11_D041d30_393660\x94\x8c\x11plate11_M099d0_20\x94\x8c\x11plate11_M099d0_60\x94\x8c\x12plate11_M099d0_180\x94\x8c\x12plate11_M099d0_540\x94\x8c\x13plate11_M099d0_1620\x94\x8c\x13plate11_M099d0_4860\x94\x8c\x14plate11_M099d0_14580\x94\x8c\x14plate11_M099d0_43740\x94\x8c\x15plate11_M099d0_131220\x94\x8c\x15plate11_M099d0_393660\x94\x8c\x13plate11_M099d182_20\x94\x8c\x13plate11_M099d182_60\x94\x8c\x14plate11_M099d182_180\x94\x8c\x14plate11_M099d182_540\x94\x8c\x15plate11_M099d182_1620\x94\x8c\x15plate11_M099d182_4860\x94\x8c\x16plate11_M099d182_14580\x94\x8c\x16plate11_M099d182_43740\x94\x8c\x17plate11_M099d182_131220\x94\x8c\x17plate11_M099d182_393660\x94\x8c\x12plate11_M099d30_20\x94\x8c\x12plate11_M099d30_60\x94\x8c\x13plate11_M099d30_180\x94\x8c\x13plate11_M099d30_540\x94\x8c\x14plate11_M099d30_1620\x94\x8c\x14plate11_M099d30_4860\x94\x8c\x15plate11_M099d30_14580\x94\x8c\x15plate11_M099d30_43740\x94\x8c\x16plate11_M099d30_131220\x94\x8c\x16plate11_M099d30_393660\x94\x8c\x11plate11_Y044d0_20\x94\x8c\x11plate11_Y044d0_60\x94\x8c\x12plate11_Y044d0_180\x94\x8c\x12plate11_Y044d0_540\x94\x8c\x13plate11_Y044d0_1620\x94\x8c\x13plate11_Y044d0_4860\x94\x8c\x14plate11_Y044d0_14580\x94\x8c\x14plate11_Y044d0_43740\x94\x8c\x15plate11_Y044d0_131220\x94\x8c\x15plate11_Y044d0_393660\x94\x8c\x12plate11_Y044d30_20\x94\x8c\x12plate11_Y044d30_60\x94\x8c\x13plate11_Y044d30_180\x94\x8c\x13plate11_Y044d30_540\x94\x8c\x14plate11_Y044d30_1620\x94\x8c\x14plate11_Y044d30_4860\x94\x8c\x15plate11_Y044d30_14580\x94\x8c\x15plate11_Y044d30_43740\x94\x8c\x16plate11_Y044d30_131220\x94\x8c\x16plate11_Y044d30_393660\x94\x8c\x0eplate11_none-2\x94\x8c\x0eplate11_none-3\x94\x8c\x0eplate11_none-4\x94\x8c\x0eplate11_none-5\x94\x8c\x0eplate11_none-6\x94\x8c\x0eplate11_none-7\x94\x8c\x0eplate11_none-8\x94\x8c\x0eplate11_none-9\x94\x8c\x0fplate11_none-10\x94\x8c\x0fplate11_none-11\x94\x8c\x0fplate11_none-12\x94\x8c\x0fplate11_none-13\x94\x8c\x0fplate11_none-14\x94\x8c\x0fplate11_none-15\x94\x8c\x0fplate11_none-16\x94\x8c\x0fplate11_none-17\x94et\x94ba]\x94j)\x01\x00\x00j,\x01\x00\x00}\x94(j.\x01\x00\x00Nj/\x01\x00\x00K\x00j0\x01\x00\x00K`j1\x01\x00\x00K\x01u\x86\x94R\x94a}\x94\x8c\x060.14.1\x94}\x94(\x8c\x04axes\x94j&\x01\x00\x00\x8c\x06blocks\x94]\x94}\x94(\x8c\x06values\x94j>\x01\x00\x00\x8c\x08mgr_locs\x94\x8c\x08builtins\x94\x8c\x05slice\x94\x93\x94K\x00K`K\x01\x87\x94R\x94uaust\x94b\x8c\x04_typ\x94\x8c\x06series\x94\x8c\t_metadata\x94]\x94h\xe3a\x8c\x05attrs\x94}\x94\x8c\x06_flags\x94}\x94\x8c\x17allows_duplicate_labels\x94\x88sh\xe3\x8c\x06sample\x94ub}\x94(\x8c\x04date\x94\x8c\n2023-09-26\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate11_samples.csv\x94\x8c\x07samples\x94\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(j!\x01\x00\x00j"\x01\x00\x00\x8c\x0cBlockManager\x94\x93\x94(\x8c\x16pandas._libs.internals\x94\x8c\x0f_unpickle_block\x94\x93\x94j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K\x02K`\x86\x94jD\x01\x00\x00\x89]\x94(\x8c\x06D041d0\x94j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00\x8c\x08D041d182\x94j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00\x8c\x07D041d30\x94j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00\x8c\x06M099d0\x94j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00\x8c\x08M099d182\x94j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00\x8c\x07M099d30\x94j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00\x8c\x06Y044d0\x94j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00\x8c\x07Y044d30\x94j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00\x8c\x04none\x94j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00j\xe7\x01\x00\x00\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc1_S16_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc2_S24_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc3_S32_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc4_S40_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc5_S48_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc6_S56_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc7_S64_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc8_S72_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc9_S80_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d0_conc10_S88_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc1_S14_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc2_S22_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc3_S30_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc4_S38_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc5_S46_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc6_S54_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc7_S62_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc8_S70_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc9_S78_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d182_conc10_S86_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc1_S15_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc2_S23_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc3_S31_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc4_S39_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc5_S47_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc6_S55_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc7_S63_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc8_S71_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc9_S79_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/D041_d30_conc10_S87_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc1_S11_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc2_S19_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc3_S27_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc4_S35_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc5_S43_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc6_S51_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc7_S59_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc8_S67_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc9_S75_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d0_conc10_S83_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc1_S9_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc2_S17_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc3_S25_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc4_S33_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc5_S41_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc6_S49_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc7_S57_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc8_S65_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc9_S73_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d182_conc10_S81_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc1_S10_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc2_S18_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc3_S26_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc4_S34_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc5_S42_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc6_S50_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc7_S58_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc8_S66_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc9_S74_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/M099_d30_conc10_S82_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc1_S13_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc2_S21_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc3_S29_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc4_S37_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc5_S45_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc6_S53_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc7_S61_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc8_S69_R1_001.fastq.gz\x94\x8c}/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc9_S77_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d0_conc10_S85_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc1_S12_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc2_S20_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc3_S28_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc4_S36_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc5_S44_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc6_S52_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc7_S60_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc8_S68_R1_001.fastq.gz\x94\x8c~/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc9_S76_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Y044_d30_conc10_S84_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum10_S90_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum11_S91_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum12_S92_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum13_S93_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum14_S94_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum15_S95_R1_001.fastq.gz\x94\x8c\x81/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum16_S96_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum1_S1_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum2_S2_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum3_S3_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum4_S4_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum5_S5_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum6_S6_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum7_S7_R1_001.fastq.gz\x94\x8c\x7f/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum8_S8_R1_001.fastq.gz\x94\x8c\x80/fh/fast/bloom_j/SR/ngs/illumina/aloes/230926_VH01189_167_AACWJF3M5/Unaligned/Project_aloes/Plate11_Noserum9_S89_R1_001.fastq.gz\x94et\x94bj\xb8\x01\x00\x00K\x00K\x06K\x03\x87\x94R\x94K\x02\x87\x94R\x94j\xd9\x01\x00\x00\x8c\x1apandas.core.arrays.integer\x94\x8c\x0cIntegerArray\x94\x93\x94)\x81\x94}\x94(\x8c\x05_data\x94j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94jA\x01\x00\x00\x8c\x02i8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x00\x03\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00<\x00\x00\x00\x00\x00\x00\x00\xb4\x00\x00\x00\x00\x00\x00\x00\x1c\x02\x00\x00\x00\x00\x00\x00T\x06\x00\x00\x00\x00\x00\x00\xfc\x12\x00\x00\x00\x00\x00\x00\xf48\x00\x00\x00\x00\x00\x00\xdc\xaa\x00\x00\x00\x00\x00\x00\x94\x00\x02\x00\x00\x00\x00\x00\xbc\x01\x06\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x94t\x94b\x8c\x05_mask\x94j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94jA\x01\x00\x00\x8c\x02b1\x94\x89\x88\x87\x94R\x94(K\x03jE\x01\x00\x00NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89C`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x94t\x94b\x8c\x06_cache\x94}\x94\x8c\x05dtype\x94jM\x02\x00\x00\x8c\nInt64Dtype\x94\x93\x94)\x81\x94}\x94ji\x02\x00\x00}\x94(\x8c\x0bnumpy_dtype\x94jY\x02\x00\x00\x8c\x04kind\x94\x8c\x01i\x94usbsubj\xb8\x01\x00\x00K\x01K\x02K\x01\x87\x94R\x94K\x02\x87\x94R\x94j\xd9\x01\x00\x00jO\x02\x00\x00)\x81\x94}\x94(jR\x02\x00\x00j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94jY\x02\x00\x00\x89B\x00\x03\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00\x00\x00\x00\x00\r\x00\x00\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x00\x00\x00\x00\x00\x94t\x94bj^\x02\x00\x00j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K`\x85\x94je\x02\x00\x00\x89C`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x94t\x94bji\x02\x00\x00}\x94jk\x02\x00\x00jn\x02\x00\x00subj\xb8\x01\x00\x00K\x02K\x03K\x01\x87\x94R\x94K\x02\x87\x94R\x94j\xd9\x01\x00\x00j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94jD\x01\x00\x00\x89]\x94(j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xdf\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe0\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe1\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe2\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe3\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe4\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe5\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00j\xe6\x01\x00\x00\x8c\x06none-2\x94\x8c\x06none-3\x94\x8c\x06none-4\x94\x8c\x06none-5\x94\x8c\x06none-6\x94\x8c\x06none-7\x94\x8c\x06none-8\x94\x8c\x06none-9\x94\x8c\x07none-10\x94\x8c\x07none-11\x94\x8c\x07none-12\x94\x8c\x07none-13\x94\x8c\x07none-14\x94\x8c\x07none-15\x94\x8c\x07none-16\x94\x8c\x07none-17\x94et\x94bj\xb8\x01\x00\x00K\x04K\x05K\x01\x87\x94R\x94K\x02\x87\x94R\x94j\xd9\x01\x00\x00j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94jD\x01\x00\x00\x89]\x94(\x8c\tD041d0_20\x94\x8c\tD041d0_60\x94\x8c\nD041d0_180\x94\x8c\nD041d0_540\x94\x8c\x0bD041d0_1620\x94\x8c\x0bD041d0_4860\x94\x8c\x0cD041d0_14580\x94\x8c\x0cD041d0_43740\x94\x8c\rD041d0_131220\x94\x8c\rD041d0_393660\x94\x8c\x0bD041d182_20\x94\x8c\x0bD041d182_60\x94\x8c\x0cD041d182_180\x94\x8c\x0cD041d182_540\x94\x8c\rD041d182_1620\x94\x8c\rD041d182_4860\x94\x8c\x0eD041d182_14580\x94\x8c\x0eD041d182_43740\x94\x8c\x0fD041d182_131220\x94\x8c\x0fD041d182_393660\x94\x8c\nD041d30_20\x94\x8c\nD041d30_60\x94\x8c\x0bD041d30_180\x94\x8c\x0bD041d30_540\x94\x8c\x0cD041d30_1620\x94\x8c\x0cD041d30_4860\x94\x8c\rD041d30_14580\x94\x8c\rD041d30_43740\x94\x8c\x0eD041d30_131220\x94\x8c\x0eD041d30_393660\x94\x8c\tM099d0_20\x94\x8c\tM099d0_60\x94\x8c\nM099d0_180\x94\x8c\nM099d0_540\x94\x8c\x0bM099d0_1620\x94\x8c\x0bM099d0_4860\x94\x8c\x0cM099d0_14580\x94\x8c\x0cM099d0_43740\x94\x8c\rM099d0_131220\x94\x8c\rM099d0_393660\x94\x8c\x0bM099d182_20\x94\x8c\x0bM099d182_60\x94\x8c\x0cM099d182_180\x94\x8c\x0cM099d182_540\x94\x8c\rM099d182_1620\x94\x8c\rM099d182_4860\x94\x8c\x0eM099d182_14580\x94\x8c\x0eM099d182_43740\x94\x8c\x0fM099d182_131220\x94\x8c\x0fM099d182_393660\x94\x8c\nM099d30_20\x94\x8c\nM099d30_60\x94\x8c\x0bM099d30_180\x94\x8c\x0bM099d30_540\x94\x8c\x0cM099d30_1620\x94\x8c\x0cM099d30_4860\x94\x8c\rM099d30_14580\x94\x8c\rM099d30_43740\x94\x8c\x0eM099d30_131220\x94\x8c\x0eM099d30_393660\x94\x8c\tY044d0_20\x94\x8c\tY044d0_60\x94\x8c\nY044d0_180\x94\x8c\nY044d0_540\x94\x8c\x0bY044d0_1620\x94\x8c\x0bY044d0_4860\x94\x8c\x0cY044d0_14580\x94\x8c\x0cY044d0_43740\x94\x8c\rY044d0_131220\x94\x8c\rY044d0_393660\x94\x8c\nY044d30_20\x94\x8c\nY044d30_60\x94\x8c\x0bY044d30_180\x94\x8c\x0bY044d30_540\x94\x8c\x0cY044d30_1620\x94\x8c\x0cY044d30_4860\x94\x8c\rY044d30_14580\x94\x8c\rY044d30_43740\x94\x8c\x0eY044d30_131220\x94\x8c\x0eY044d30_393660\x94j\x90\x02\x00\x00j\x91\x02\x00\x00j\x92\x02\x00\x00j\x93\x02\x00\x00j\x94\x02\x00\x00j\x95\x02\x00\x00j\x96\x02\x00\x00j\x97\x02\x00\x00j\x98\x02\x00\x00j\x99\x02\x00\x00j\x9a\x02\x00\x00j\x9b\x02\x00\x00j\x9c\x02\x00\x00j\x9d\x02\x00\x00j\x9e\x02\x00\x00j\x9f\x02\x00\x00et\x94bj\xb8\x01\x00\x00K\x05K\x06K\x01\x87\x94R\x94K\x02\x87\x94R\x94j\xd9\x01\x00\x00j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94jD\x01\x00\x00\x89]\x94(jH\x01\x00\x00jI\x01\x00\x00jJ\x01\x00\x00jK\x01\x00\x00jL\x01\x00\x00jM\x01\x00\x00jN\x01\x00\x00jO\x01\x00\x00jP\x01\x00\x00jQ\x01\x00\x00jR\x01\x00\x00jS\x01\x00\x00jT\x01\x00\x00jU\x01\x00\x00jV\x01\x00\x00jW\x01\x00\x00jX\x01\x00\x00jY\x01\x00\x00jZ\x01\x00\x00j[\x01\x00\x00j\\\x01\x00\x00j]\x01\x00\x00j^\x01\x00\x00j_\x01\x00\x00j`\x01\x00\x00ja\x01\x00\x00jb\x01\x00\x00jc\x01\x00\x00jd\x01\x00\x00je\x01\x00\x00jf\x01\x00\x00jg\x01\x00\x00jh\x01\x00\x00ji\x01\x00\x00jj\x01\x00\x00jk\x01\x00\x00jl\x01\x00\x00jm\x01\x00\x00jn\x01\x00\x00jo\x01\x00\x00jp\x01\x00\x00jq\x01\x00\x00jr\x01\x00\x00js\x01\x00\x00jt\x01\x00\x00ju\x01\x00\x00jv\x01\x00\x00jw\x01\x00\x00jx\x01\x00\x00jy\x01\x00\x00jz\x01\x00\x00j{\x01\x00\x00j|\x01\x00\x00j}\x01\x00\x00j~\x01\x00\x00j\x7f\x01\x00\x00j\x80\x01\x00\x00j\x81\x01\x00\x00j\x82\x01\x00\x00j\x83\x01\x00\x00j\x84\x01\x00\x00j\x85\x01\x00\x00j\x86\x01\x00\x00j\x87\x01\x00\x00j\x88\x01\x00\x00j\x89\x01\x00\x00j\x8a\x01\x00\x00j\x8b\x01\x00\x00j\x8c\x01\x00\x00j\x8d\x01\x00\x00j\x8e\x01\x00\x00j\x8f\x01\x00\x00j\x90\x01\x00\x00j\x91\x01\x00\x00j\x92\x01\x00\x00j\x93\x01\x00\x00j\x94\x01\x00\x00j\x95\x01\x00\x00j\x96\x01\x00\x00j\x97\x01\x00\x00j\x98\x01\x00\x00j\x99\x01\x00\x00j\x9a\x01\x00\x00j\x9b\x01\x00\x00j\x9c\x01\x00\x00j\x9d\x01\x00\x00j\x9e\x01\x00\x00j\x9f\x01\x00\x00j\xa0\x01\x00\x00j\xa1\x01\x00\x00j\xa2\x01\x00\x00j\xa3\x01\x00\x00j\xa4\x01\x00\x00j\xa5\x01\x00\x00j\xa6\x01\x00\x00j\xa7\x01\x00\x00et\x94bj\xb8\x01\x00\x00K\x06K\x07K\x01\x87\x94R\x94K\x02\x87\x94R\x94j\xd9\x01\x00\x00j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K\x01K`\x86\x94jD\x01\x00\x00\x89]\x94(\x8c\x07plate11\x94j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00j\x0e\x03\x00\x00et\x94bj\xb8\x01\x00\x00K\x07K\x08K\x01\x87\x94R\x94K\x02\x87\x94R\x94t\x94]\x94(j)\x01\x00\x00j\'\x01\x00\x00\x8c\x05Index\x94\x93\x94}\x94(\x8c\x04data\x94j7\x01\x00\x00j:\x01\x00\x00K\x00\x85\x94j<\x01\x00\x00\x87\x94R\x94(K\x01K\x08\x85\x94jD\x01\x00\x00\x89]\x94(\x8c\x05serum\x94\x8c\x0fdilution_factor\x94\x8c\treplicate\x94\x8c\x05fastq\x94\x8c\x0fserum_replicate\x94\x8c\x0esample_noplate\x94j\xc5\x01\x00\x00\x8c\x05plate\x94et\x94bj.\x01\x00\x00Nu\x86\x94R\x94j3\x01\x00\x00e\x86\x94R\x94j\xbc\x01\x00\x00\x8c\tdataframe\x94j\xbe\x01\x00\x00]\x94j\xc0\x01\x00\x00}\x94j\xc2\x01\x00\x00}\x94j\xc4\x01\x00\x00\x88subu}\x94(\x8c\x12avg_barcode_counts\x94M\xf4\x01\x8c\x16min_neut_standard_frac\x94G?\x94z\xe1G\xae\x14{\x8c\x1fmax_neut_standard_frac_no_serum\x94G?\xb9\x99\x99\x99\x99\x99\x9aue}\x94(h\xcd}\x94(j\xcf\x01\x00\x00K\x00N\x86\x94\x8c\x0cplate_params\x94K\x01N\x86\x94\x8c\rqc_thresholds\x94K\x02N\x86\x94uh\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bj\xcf\x01\x00\x00j\x1f\x01\x00\x00j6\x03\x00\x00j\xc6\x01\x00\x00j8\x03\x00\x00j/\x03\x00\x00ub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x07plate11\x94a}\x94(h\xcd}\x94\x8c\x05plate\x94K\x00N\x86\x94sh\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bj%\x03\x00\x00jG\x03\x00\x00ub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x15/loc/scratch/34309988\x94e}\x94(h\xcd}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bj]\x03\x00\x00K\x01j_\x03\x00\x00K\x01ja\x03\x00\x00jZ\x03\x00\x00ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c,results/plates/plate11/process_plate11.ipynb\x94a}\x94(h\xcd}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\xd7]\x94(h\xd9h\xdaeh\xd9h\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xd9sNt\x94bh\xdah\xddh\xdf\x85\x94R\x94(h\xdf)}\x94h\xe3h\xdasNt\x94bjs\x03\x00\x00jp\x03\x00\x00ub\x8c\x06config\x94}\x94(\x8c\x10seqneut-pipeline\x94\x8c\x10seqneut-pipeline\x94\x8c\x06plates\x94}\x94(\x8c\x06plate1\x94}\x94(\x8c\x04date\x94\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe7\x08\x01\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate1_samples.csv\x94u\x8c\x06plate2\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x01\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate2_samples.csv\x94u\x8c\x06plate3\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x02\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate3_samples.csv\x94u\x8c\x06plate4\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x02\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate4_samples.csv\x94u\x8c\x06plate5\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x04\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate5_samples.csv\x94u\x8c\x06plate6\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x04\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate6_samples.csv\x94u\x8c\x06plate7\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x05\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate7_samples.csv\x94u\x8c\x06plate8\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x05\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate8_samples.csv\x94u\x8c\x06plate9\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x05\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1edata/plates/plate9_samples.csv\x94u\x8c\x07plate10\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\x08\x06\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate10_samples.csv\x94uj\x0e\x03\x00\x00}\x94(j\xc7\x01\x00\x00j\x89\x03\x00\x00C\x04\x07\xe7\t\x1a\x94\x85\x94R\x94j\xc9\x01\x00\x00j\xca\x01\x00\x00j\xcb\x01\x00\x00j\xcc\x01\x00\x00j\xcd\x01\x00\x00j\xce\x01\x00\x00u\x8c\x07plate12\x94}\x94(\x8c\x04date\x94j\x89\x03\x00\x00C\x04\x07\xe7\t\x1a\x94\x85\x94R\x94\x8c\rviral_library\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c\x11neut_standard_set\x94\x8c\x08loes2023\x94\x8c\x0bsamples_csv\x94\x8c\x1fdata/plates/plate12_samples.csv\x94uu\x8c\x0fviral_libraries\x94}\x94\x8c\x14pdmH1N1_lib2023_loes\x94\x8c-data/viral_libraries/pdmH1N1_lib2023_loes.csv\x94s\x8c\x12neut_standard_sets\x94}\x94\x8c\x08loes2023\x94\x8c3data/neut_standard_sets/loes2023_neut_standards.csv\x94s\x8c\x1eillumina_barcode_parser_params\x94}\x94(\x8c\x08upstream\x94\x8c\x1fCTCCCTACAATGTCGGATTTGTATTTAATAG\x94\x8c\ndownstream\x94\x8c\x00\x94\x8c\x04minq\x94K\x14\x8c\x11upstream_mismatch\x94K\x04\x8c\x0ebc_orientation\x94\x8c\x02R2\x94u\x8c\x1bprocess_plate_qc_thresholds\x94j/\x03\x00\x00u\x8c\x04rule\x94\x8c\rprocess_plate\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8ct/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax/seqneut-pipeline/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2023/flu_seqneut_DRIVE_2021-22_repeat_vax');
######## snakemake preamble end #########


In [3]:
count_csvs = snakemake.input.count_csvs
fate_csvs = snakemake.input.fate_csvs
viral_library_csv = snakemake.input.viral_library_csv
neut_standard_set_csv = snakemake.input.neut_standard_set_csv
frac_infectivity_csv = snakemake.output.frac_infectivity_csv
qc_failures_file = snakemake.output.qc_failures
samples = snakemake.params.samples
plate_params = snakemake.params.plate_params
qc_thresholds = snakemake.params.qc_thresholds
plate = snakemake.wildcards.plate

samples_df = plate_params["samples"]

assert len(samples) == len(count_csvs) == len(fate_csvs) == len(samples_df)

print(f"Processing {plate=}")

# define set of QC failures for this plate
qc_failures = set([])

Processing plate='plate11'


## Statistics on barcode-parsing for each sample
Make interactive chart of the "fates" of the sequencing reads parsed for each sample on the plate.

If most sequencing reads are not "valid barcodes", this could potentially indicate some problem in the sequencing or barcode set you are parsing.

Potential fates are:
 - *valid barcode*: barcode that matches a known virus or neutralization standard, we hope most reads are this.
 - *invalid barcode*: a barcode with proper flanking sequences, but does not match a known virus or neutralization standard. If you  have a lot of reads of this type, it is probably a good idea to look at the invalid barcode CSVs (in the `./results/barcode_invalid/` subdirectory created by the pipeline) to see what these invalid barcodes are.
 - *unparseable barcode*: could not parse a barcode from this read as there was not a sequence of the correct length with the appropriate flanking sequence.
 - *low quality barcode*: low-quality or `N` nucleotides in barcode, could indicate problem with sequencing.
 - *failed chastity filter*: reads that failed the Illumina chastity filter, if these are reported in the FASTQ (they may not be).

Also, if the number of reads per sample is very uneven, that could indicate that you did not do a good job of balancing the different samples in the Illumina sequencing.

In [4]:
fates = (
    pd.concat([pd.read_csv(f).assign(sample=s) for f, s in zip(fate_csvs, samples)])
    .merge(samples_df, validate="many_to_one", on="sample")
    .assign(fate_counts=lambda x: x.groupby("fate")["count"].transform("sum"))
    .query("fate_counts > 0")  # only keep fates with at least one count
    [["fate", "count", "serum", "sample_noplate", "dilution_factor"]]
)

assert len(fates) == len(fates.drop_duplicates())

serum_selection = alt.selection_point(
    fields=["serum"],
    bind=alt.binding_select(
        options=[None] + sorted(fates["serum"].unique().tolist()),
        labels=["all"] + [str(s) for s in sorted(fates["serum"].unique())],
        name="serum",
    )
)   

fates_chart = (
    alt.Chart(fates)
    .add_params(serum_selection)
    .transform_filter(serum_selection)
    .encode(
        alt.X("count", scale=alt.Scale(nice=False, padding=3)),
        alt.Y(
            "sample_noplate",
            title=None,
            sort=list(
                fates.sort_values(["serum", "dilution_factor"])["sample_noplate"]
            ),
        ),
        alt.Color("fate", sort=sorted(fates["fate"].unique(), reverse=True)),
        alt.Order("fate", sort="descending"),
        tooltip=fates.columns.tolist(),
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=200,
        title=f"Barcode parsing for {plate}",
    )
    .configure_axis(grid=False)
)

fates_chart

alt.Chart(...)

## Counts per barcode

First get the counts per barcode and classification information on these barcodes:

In [5]:
# get barcode counts
counts = (
    pd.concat([pd.read_csv(c).assign(sample=s) for c, s in zip(count_csvs, samples)])
    .merge(samples_df, validate="many_to_one", on="sample")
    .drop(columns=["replicate", "plate", "fastq"])
)

# get classification of barcodes as viral or neut standard
barcode_class = pd.concat(
    [
        pd.read_csv(viral_library_csv)[["barcode", "strain"]].assign(
            neut_standard=False,
        ),
        pd.read_csv(neut_standard_set_csv)[["barcode"]].assign(
            neut_standard=True, strain=pd.NA,
        ),
    ],
    ignore_index=True
)

# merge counts and classification of barcodes
assert set(counts["barcode"]) == set(barcode_class["barcode"])
counts = counts.merge(barcode_class, on="barcode", validate="many_to_one")

Plot average counts per barcode, and make sure that these pass the QC threshold.
If a sample has inadequate barcode counts, it may not have good enough statistics for accurate analysis:

In [16]:
avg_barcode_counts = (
    counts
    .groupby(
        ["serum", "dilution_factor", "sample_noplate"],
        dropna=False,
        as_index=False,
    )
    .aggregate(avg_count=pd.NamedAgg("count", "mean"))
    .assign(passes_qc=lambda x: x["avg_count"] >= qc_thresholds["avg_barcode_counts"])
)

avg_barcode_counts_chart = (
    alt.Chart(avg_barcode_counts)
    .add_params(serum_selection)
    .transform_filter(serum_selection)
    .encode(
        alt.X(
            "avg_count",
            title="average counts per barcode",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y(
            "sample_noplate",
            title=None,
            sort=list(
                avg_barcode_counts.sort_values(["serum", "dilution_factor"])["sample_noplate"]
            ),
        ),
        alt.Color(
            "passes_qc",
            title=f"passes QC threshold {qc_thresholds['avg_barcode_counts']}",
            scale=alt.Scale(domain=[True, False]),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if avg_barcode_counts[c].dtype == float
            else c
            for c in avg_barcode_counts.columns
        ],
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=200,
        title=f"Average barcode counts for {plate}",
    )
    .configure_axis(grid=False)
)

display(avg_barcode_counts_chart)

if not avg_barcode_counts["passes_qc"].all():
    qc_failures.add("avg_barcode_counts")
    print(f"\nThe following samples failed {qc_thresholds['avg_barcode_counts']=}")
    display(avg_barcode_counts.query("not passes_qc").reset_index(drop=True))
else:
    print(f"\nAll samples passed {qc_thresholds['avg_barcode_counts']=}")

alt.Chart(...)


All samples passed qc_thresholds['avg_barcode_counts']=500


## Fraction of counts from neutralization standard
Determine the fraction of counts from the neutralization standard in each sample.

In [15]:
neut_standard_fracs = (
    counts
    .assign(neut_standard_count=lambda x: x["count"] * x["neut_standard"].astype(int))
    .groupby(
        ["serum", "dilution_factor", "sample_noplate"],
        dropna=False,
        as_index=False,
    )
    .aggregate(
        total_count=pd.NamedAgg("count", "sum"),
        neut_standard_count=pd.NamedAgg("neut_standard_count", "sum"),
    )
    .assign(
        neut_standard_frac=lambda x: x["neut_standard_count"] / x["total_count"],
        passes_qc=lambda x: (
            (x["neut_standard_frac"] >= qc_thresholds["min_neut_standard_frac"])
            & (
                (x["serum"] != "none")
                | (x["neut_standard_frac"] <= qc_thresholds["max_neut_standard_frac_no_serum"])
            )
        ),
    )
)

neut_standard_qc_desc = (
    f"neut standard frac >= {qc_thresholds['min_neut_standard_frac']}, "
    + f"<= {qc_thresholds['max_neut_standard_frac_no_serum']} for no-serum samples"
)

neut_standard_fracs_chart = (
    alt.Chart(neut_standard_fracs)
    .add_params(serum_selection)
    .transform_filter(serum_selection)
    .encode(
        alt.X(
            "neut_standard_frac",
            title="fraction of counts from neutralization standard",
            scale=alt.Scale(nice=False, padding=3),
        ),
        alt.Y(
            "sample_noplate",
            title=None,
            sort=list(
                neut_standard_fracs.sort_values(["serum", "dilution_factor"])["sample_noplate"]
            ),
        ),
        alt.Color(
            "passes_qc",
            title=neut_standard_qc_desc,
            scale=alt.Scale(domain=[True, False]),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if neut_standard_fracs[c].dtype == float
            else c
            for c in neut_standard_fracs.columns
        ],
    )
    .mark_bar(height={"band": 0.85})
    .properties(
        height=alt.Step(10),
        width=200,
        title=f"Neutralization-standard fractions for {plate}",
    )
    .configure_axis(grid=False)
    .configure_legend(titleLimit=1000)
)

display(neut_standard_fracs_chart)

if not neut_standard_fracs["passes_qc"].all():
    qc_failures.add("min_neut_standard_frac or max_neut_standard_frac_no_serum")
    print(f"\nThe following samples failed {neut_standard_qc_desc}")
    display(neut_standard_fracs.query("not passes_qc").reset_index(drop=True))
else:
    print(f"\nAll samples passed {neut_standard_qc_desc}")

alt.Chart(...)


The following samples failed neut standard frac >= 0.02, <= 0.1 for no-serum samples


,serum,dilution_factor,sample_noplate,total_count,neut_standard_count,neut_standard_frac,passes_qc
0,D041d0,4860,D041d0_4860,1021571,19001,0.018600,False
1,D041d182,4860,D041d182_4860,1720382,26756,0.015552,False
2,D041d182,43740,D041d182_43740,1844014,21399,0.011605,False
3,M099d0,4860,M099d0_4860,1500619,26708,0.017798,False
4,M099d0,43740,M099d0_43740,1940012,23207,0.011962,False
5,M099d0,131220,M099d0_131220,1497969,26471,0.017671,False
6,Y044d30,43740,Y044d30_43740,1984070,24895,0.012547,False
7,Y044d30,131220,Y044d30_131220,2226172,36237,0.016278,False
8,none,<NA>,none-11,1514580,28467,0.018795,False
9,none,<NA>,none-17,1794001,34228,0.019079,False


## Evenness

In [50]:
barcode_selection = alt.selection_point(fields=["barcode"], on="mouseover", empty=False)

for is_neut_standard, df in counts.groupby("neut_standard"):
    if not is_neut_standard:
        df = df.query("serum == 'none'")
    df = (
        df.assign(
            sample_counts=lambda x: x.groupby("sample")["count"].transform("sum"),
            count_frac=lambda x: x["count"] / x["sample_counts"],
        )
        .drop(
            columns=(
                ["sample", "serum_replicate", "sample_counts", "neut_standard"]
                + (["strain"] if is_neut_standard else ["dilution_factor"])
            ),
        )
    )

    evenness_chart = (
        alt.Chart(df)
        .add_params(barcode_selection)
        .encode(
            alt.X(
                "count_frac",
                title=(
                    "barcode's fraction of neut standard counts"
                    if is_neut_standard
                    else "barcode's fraction of non-neut standard counts"
                ),
                scale=alt.Scale(nice=False, padding=5),
            ),
            alt.Y(
                "sample_noplate",
                title=None,
                sort=list(
                    neut_standard_fracs.sort_values(["serum", "dilution_factor"])["sample_noplate"]
                ),
            ),
            alt.Fill("barcode", legend=None),
            strokeWidth=alt.condition(barcode_selection, alt.value(2), alt.value(0)),
            size=alt.condition(barcode_selection, alt.value(60), alt.value(35)),
            tooltip=[
                alt.Tooltip(c, format=".3g") if df[c].dtype == float
                else c
                for c in df.columns
            ],
        )
        .mark_circle(fillOpacity=0.6, stroke="black", strokeOpacity=1)
        .properties(
            height=alt.Step(10),
            width=300,
            title=(
                f"{plate} all samples, neut-standard barcodes"
                if is_neut_standard
                else f"{plate} no-serum samples, all barcodes"
            ),
        )
        .configure_axis(grid=False)
        .configure_legend(titleLimit=1000)
        .interactive()
    )

    if is_neut_standard:
        evenness_chart = evenness_chart.add_params(serum_selection).transform_filter(
            serum_selection
        )

    display(evenness_chart)

alt.Chart(...)

alt.Chart(...)